In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from utils import chart_price, plot, perf, v_backtester, c_backtester, perf_var, summary
import sys
sys.path.append('/home/tomek/ib_tools')
import matplotlib.pyplot as plt
%matplotlib inline
from tester import run, simulate, calibrate, calibrate_multiple

In [2]:
def ema(data, periods, smooth, vol):
    data = data.copy()
    data['ema_fast'] = data.close.ewm(span=periods).mean()
    data['ema_slow'] = data.close.ewm(span=periods*3).mean()
    data['ema_diff'] = data.ema_fast - data.ema_slow
    data['ema_diff_norm'] = data.ema_diff / vol 
    #data.ema_diff.ewm(span=vol_lookback).std()
    #data['ema_diff_norm'] = data.ema_diff / data.ema_diff.rolling(periods).std()
    #data[f'ema_{periods}'] = data['ema_diff_norm'].ewm(span=smooth).mean()
    data[f'ema_{periods}'] = data['ema_diff_norm']
    return data[f'ema_{periods}']

In [3]:
def reducer(x):
    if x < -2.7:
        return max(-(np.log(np.abs(x) -2.7) - .5), 0)
    elif x > 2.7:
        return min((np.log(np.abs(x) -2.7) - .5), 0)
    else:
        return - x**3

In [4]:
def bolli(data, periods, smooth, vol):
    data = data.copy()
    data['mid'] = data.close.ewm(span=periods).mean()
    data['distance'] = data.mid - data.close
    data['std'] = data.close.ewm(span=periods).std()
    data['distance_scaled'] = data.distance / data['std']
    data[f'bolli_{periods}'] = data['distance_scaled']
    return data[f'bolli_{periods}']

In [5]:
def bolli_reduced(data, periods, smooth, vol):
    data = data.copy()
    #data[f'bolli_reduced_{periods}'] = smooth(bolli(data, periods, smooth, vol).apply(reducer), periods/3)
    data[f'bolli_reduced_{periods}'] = bolli(data, periods, smooth, vol).apply(reducer)
    return data[f'bolli_reduced_{periods}']

In [6]:
def breakout(data, periods, smooth, vol):
    up  = (data.close >= data.close.rolling(periods).max()) * 1
    down = (data.close <= data.close.rolling(periods).min()) * 1
    return (up - down).ewm(span=max((int(periods/3), 1))).mean().rename(f'break_{periods}', inplace=True)

In [7]:
def carver(data, periods, smooth, vol):
    data = data.copy()
    data['max'] = data.close.rolling(periods).max()
    data['min'] = data.close.rolling(periods).min()
    data['mid'] = data[['min', 'max']].mean(axis=1)
    data['breakout'] = data['close'] - data['mid']
    data['breakout_norm'] = data['breakout'] / (data['max'] - data['min'])
    data[f'carver_{periods}'] = smooth(data['breakout_norm'], periods/3)
    return data[f'carver_{periods}']

In [8]:
def roc(data, periods, smooth, vol):
    data = data.copy()
    data['roc'] = data.close.pct_change(periods) * 1000
    #data['std'] = data['roc'].ewm(span=200).std()
    #data['roc_normalized'] = data['roc'] / data['std']
    data['roc_normalized'] = data['roc'] / vol
    #data[f'roc_{periods}'] = smooth(data['roc_normalized'], periods/3)
    data[f'roc_{periods}'] = data['roc_normalized']
    return data[f'roc_{periods}']

In [50]:
data = calibrate_multiple('NQ', breakout, start_date='20180501')

In [51]:
data[0]

break_5      0.210944
break_10     0.155669
break_20     0.139221
break_40     0.138401
break_80     0.152160
break_160    0.203605
dtype: float64

In [52]:
data[0].sum()

1.0000000000000002

In [53]:
data[1]

break_5       17.726329
break_10      27.368607
break_20      38.807890
break_40      57.384371
break_80      78.033298
break_160    149.681161
dtype: float64

In [45]:
data[2]

1.3283114373909224

In [14]:
weights = pd.DataFrame()
adjustments = pd.DataFrame()
multipliers = pd.Series()
for symbol in ['NQ', 'ES', 'GC', 'CL']:
    w,a,m = calibrate_multiple(symbol, roc, start_date='20180501')
    weights[symbol] = w
    adjustments[symbol] = a
    multipliers[symbol] = m
weights, adjustments, multipliers

(               NQ        ES        GC        CL
 roc_5    0.178715  0.179040  0.184085  0.186235
 roc_10   0.151361  0.151978  0.155997  0.158317
 roc_20   0.143719  0.143378  0.146734  0.149861
 roc_40   0.146668  0.149847  0.148184  0.150111
 roc_80   0.158702  0.158919  0.161177  0.162949
 roc_160  0.220835  0.216838  0.203822  0.192527,
                 NQ         ES        GC        CL
 roc_5    40.561747  16.189364  7.410745  0.330504
 roc_10   28.312731  11.238484  5.065478  0.229884
 roc_20   19.853591   7.884829  3.584298  0.162346
 roc_40   14.323670   5.640591  2.478450  0.116139
 roc_80    9.827076   3.897318  1.763103  0.079725
 roc_160   6.972325   2.727416  1.435119  0.055885,
 NQ    1.328311
 ES    1.329804
 GC    1.334028
 CL    1.311479
 dtype: float64)

In [15]:
1/6

0.16666666666666666

In [16]:
from tester import get_data, get_candles, get_avg_vol, get_vol

In [17]:
data = get_data('ES', '20180901', '20191231')

In [18]:
data

,open,high,low,close,volume,average,barCount
date,,,,,,,
2018-09-03 00:00:00,2906.50,2907.00,2905.75,2905.75,949,2906.375,161
2018-09-03 00:01:00,2905.75,2906.25,2905.75,2905.75,329,2906.125,80
2018-09-03 00:02:00,2905.75,2906.00,2905.75,2906.00,53,2905.900,19
2018-09-03 00:03:00,2906.25,2906.75,2906.25,2906.75,145,2906.450,49
2018-09-03 00:04:00,2906.50,2906.75,2906.00,2906.00,135,2906.300,46
...,...,...,...,...,...,...,...
2019-09-05 08:51:00,2957.50,2957.75,2957.50,2957.50,361,2957.525,70
2019-09-05 08:52:00,2957.50,2958.00,2957.00,2957.25,517,2957.575,112
2019-09-05 08:53:00,2957.50,2957.75,2957.25,2957.50,267,2957.400,85


In [19]:
months = [g for n, g in data.groupby(pd.Grouper(freq='M'))]

In [20]:
months

[                        open     high      low    close  volume   average  \
 date                                                                        
 2018-09-03 00:00:00  2906.50  2907.00  2905.75  2905.75     949  2906.375   
 2018-09-03 00:01:00  2905.75  2906.25  2905.75  2905.75     329  2906.125   
 2018-09-03 00:02:00  2905.75  2906.00  2905.75  2906.00      53  2905.900   
 2018-09-03 00:03:00  2906.25  2906.75  2906.25  2906.75     145  2906.450   
 2018-09-03 00:04:00  2906.50  2906.75  2906.00  2906.00     135  2906.300   
 ...                      ...      ...      ...      ...     ...       ...   
 2018-09-28 22:55:00  2920.75  2921.00  2920.75  2920.75      28  2920.850   
 2018-09-28 22:56:00  2920.75  2921.00  2920.75  2921.00     184  2921.000   
 2018-09-28 22:57:00  2921.00  2921.00  2920.75  2921.00      41  2920.850   
 2018-09-28 22:58:00  2920.75  2921.00  2920.75  2920.75     465  2920.900   
 2018-09-28 22:59:00  2920.75  2921.75  2920.75  2921.75     272

In [21]:
months[9]

,open,high,low,close,volume,average,barCount
date,,,,,,,
2019-06-03 00:00:00,2741.00,2744.50,2740.00,2740.00,6349,2741.625,1101
2019-06-03 00:01:00,2740.00,2741.25,2738.75,2739.75,3294,2740.150,702
2019-06-03 00:02:00,2740.00,2741.00,2739.50,2740.75,1285,2740.600,254
2019-06-03 00:03:00,2740.50,2740.75,2739.50,2740.75,732,2740.125,194
2019-06-03 00:04:00,2740.75,2741.00,2739.00,2739.25,887,2739.700,219
...,...,...,...,...,...,...,...
2019-06-28 22:55:00,2953.75,2953.75,2952.75,2953.00,280,2953.300,35
2019-06-28 22:56:00,2952.75,2953.50,2952.50,2953.00,416,2953.000,130
2019-06-28 22:57:00,2953.00,2953.50,2952.75,2953.50,303,2953.050,87


In [22]:
candles = get_candles(data, get_avg_vol(data, 120)).set_index('date')

In [23]:
candles

,open,high,low,close,barCount,volume
date,,,,,,
2018-09-04 02:31:00,2906.50,2912.50,2902.50,2904.50,34605,134452
2018-09-04 13:31:00,2904.50,2912.00,2894.75,2896.00,28532,134107
2018-09-04 15:34:00,2896.00,2898.50,2891.75,2896.00,16732,130583
2018-09-04 15:50:00,2896.00,2896.25,2886.25,2886.50,19416,144627
2018-09-04 16:18:00,2886.75,2898.25,2886.50,2896.50,17426,139397
...,...,...,...,...,...,...
2019-09-04 19:03:00,2928.00,2934.50,2927.75,2933.75,22662,136899
2019-09-04 21:07:00,2934.00,2935.50,2929.00,2935.00,20499,134818
2019-09-04 21:58:00,2935.00,2939.00,2934.50,2937.75,15157,123074


In [24]:
start = months[0].index[0]
end = months[0].index[-1]

In [25]:
start, end

(Timestamp('2018-09-03 00:00:00'), Timestamp('2018-09-28 22:59:00'))

In [26]:
period = candles.loc[start:end]

In [27]:
period

,open,high,low,close,barCount,volume
date,,,,,,
2018-09-04 02:31:00,2906.50,2912.50,2902.50,2904.50,34605,134452
2018-09-04 13:31:00,2904.50,2912.00,2894.75,2896.00,28532,134107
2018-09-04 15:34:00,2896.00,2898.50,2891.75,2896.00,16732,130583
2018-09-04 15:50:00,2896.00,2896.25,2886.25,2886.50,19416,144627
2018-09-04 16:18:00,2886.75,2898.25,2886.50,2896.50,17426,139397
...,...,...,...,...,...,...
2018-09-28 17:59:00,2921.50,2925.50,2921.25,2921.75,16542,138491
2018-09-28 19:42:00,2921.50,2923.75,2915.50,2922.75,16539,135231
2018-09-28 21:11:00,2922.75,2924.00,2917.50,2918.25,15378,133601


In [28]:
len(period.resample('D').last().dropna())

19

In [29]:
165/19

8.68421052631579

In [30]:
vols = get_vol(candles, 200)

In [31]:
vols.iloc[150:].head(50)

date
2018-09-27 16:07:00    4.681867
2018-09-27 16:58:00    4.678080
2018-09-27 18:17:00    4.680105
2018-09-27 20:06:00    4.710454
2018-09-27 21:12:00    4.682707
2018-09-27 21:58:00    4.652760
2018-09-28 09:56:00    4.651221
2018-09-28 14:51:00    4.655135
2018-09-28 15:57:00    4.656622
2018-09-28 16:46:00    4.675544
2018-09-28 17:59:00    4.647197
2018-09-28 19:42:00    4.620693
2018-09-28 21:11:00    4.613825
2018-09-28 21:43:00    4.584909
2018-09-28 21:58:00    4.556463
2018-10-01 04:06:00    4.760929
2018-10-01 14:38:00    4.740348
2018-10-01 15:51:00    4.751622
2018-10-01 16:39:00    4.726580
2018-10-01 17:57:00    4.705555
2018-10-01 19:15:00    4.689907
2018-10-01 20:45:00    4.664714
2018-10-01 21:34:00    4.680654
2018-10-01 21:58:00    4.687031
2018-10-02 09:17:00    4.702041
2018-10-02 15:29:00    4.708473
2018-10-02 16:02:00    4.683473
2018-10-02 16:51:00    4.655646
2018-10-02 18:02:00    4.684447
2018-10-02 19:46:00    4.656480
2018-10-02 21:12:00    4.702400
201

In [32]:
r= roc(period, 160, None, vols)


In [33]:
r.iloc[50:].head(50)

date
2018-09-10 17:02:00   NaN
2018-09-10 17:56:00   NaN
2018-09-10 19:45:00   NaN
2018-09-10 21:44:00   NaN
2018-09-11 01:40:00   NaN
2018-09-11 13:28:00   NaN
2018-09-11 15:38:00   NaN
2018-09-11 16:06:00   NaN
2018-09-11 16:48:00   NaN
2018-09-11 17:48:00   NaN
2018-09-11 18:42:00   NaN
2018-09-11 20:30:00   NaN
2018-09-11 21:58:00   NaN
2018-09-12 11:21:00   NaN
2018-09-12 15:36:00   NaN
2018-09-12 16:06:00   NaN
2018-09-12 16:36:00   NaN
2018-09-12 17:11:00   NaN
2018-09-12 17:30:00   NaN
2018-09-12 18:04:00   NaN
2018-09-12 19:17:00   NaN
2018-09-12 20:44:00   NaN
2018-09-12 21:58:00   NaN
2018-09-13 12:59:00   NaN
2018-09-13 15:33:00   NaN
2018-09-13 16:00:00   NaN
2018-09-13 16:21:00   NaN
2018-09-13 16:52:00   NaN
2018-09-13 17:56:00   NaN
2018-09-13 19:47:00   NaN
2018-09-13 21:37:00   NaN
2018-09-14 07:21:00   NaN
2018-09-14 15:39:00   NaN
2018-09-14 17:10:00   NaN
2018-09-14 18:29:00   NaN
2018-09-14 21:56:00   NaN
2018-09-17 15:52:00   NaN
2018-09-17 19:40:00   NaN
2018-09

In [34]:
r.dropna()

date
2018-09-28 17:59:00    1.277987
2018-09-28 19:42:00    1.999025
2018-09-28 21:11:00    1.665215
2018-09-28 21:43:00    2.342391
2018-09-28 21:58:00    1.591175
Name: roc_160, dtype: float64

In [35]:
r.tail(50)

date
2018-09-21 15:39:00         NaN
2018-09-21 16:15:00         NaN
2018-09-21 16:56:00         NaN
2018-09-21 18:12:00         NaN
2018-09-21 19:56:00         NaN
2018-09-21 21:28:00         NaN
2018-09-21 21:57:00         NaN
2018-09-21 21:59:00         NaN
2018-09-24 08:59:00         NaN
2018-09-24 15:33:00         NaN
2018-09-24 16:06:00         NaN
2018-09-24 16:39:00         NaN
2018-09-24 17:16:00         NaN
2018-09-24 18:46:00         NaN
2018-09-24 20:39:00         NaN
2018-09-24 21:55:00         NaN
2018-09-25 09:59:00         NaN
2018-09-25 15:34:00         NaN
2018-09-25 16:24:00         NaN
2018-09-25 17:26:00         NaN
2018-09-25 19:23:00         NaN
2018-09-25 20:33:00         NaN
2018-09-25 21:45:00         NaN
2018-09-25 22:09:00         NaN
2018-09-26 15:28:00         NaN
2018-09-26 16:12:00         NaN
2018-09-26 17:39:00         NaN
2018-09-26 20:00:00         NaN
2018-09-26 20:36:00         NaN
2018-09-26 21:21:00         NaN
2018-09-26 21:46:00         NaN
201

In [36]:
data = period.copy()
data['roc'] = data.close.pct_change(5) * 1000
data.head(50)

,open,high,low,close,barCount,volume,roc
date,,,,,,,
2018-09-04 02:31:00,2906.50,2912.50,2902.50,2904.50,34605,134452,NaN
2018-09-04 13:31:00,2904.50,2912.00,2894.75,2896.00,28532,134107,NaN
2018-09-04 15:34:00,2896.00,2898.50,2891.75,2896.00,16732,130583,NaN
2018-09-04 15:50:00,2896.00,2896.25,2886.25,2886.50,19416,144627,NaN
2018-09-04 16:18:00,2886.75,2898.25,2886.50,2896.50,17426,139397,NaN
2018-09-04 16:51:00,2896.75,2900.25,2893.75,2900.00,16865,135667,-1.549320
2018-09-04 17:41:00,2900.00,2901.00,2894.75,2895.25,18437,142444,-0.258978
2018-09-04 18:23:00,2895.50,2895.50,2886.00,2886.00,16157,129338,-3.453039
2018-09-04 19:40:00,2886.00,2894.25,2885.50,2892.00,18556,141710,1.905422


In [37]:
data['vol'] = vols
data['roc_normalized'] = data['roc'] / vols
data

,open,high,low,close,barCount,volume,roc,vol,roc_normalized
date,,,,,,,,,
2018-09-04 02:31:00,2906.50,2912.50,2902.50,2904.50,34605,134452,NaN,NaN,NaN
2018-09-04 13:31:00,2904.50,2912.00,2894.75,2896.00,28532,134107,NaN,NaN,NaN
2018-09-04 15:34:00,2896.00,2898.50,2891.75,2896.00,16732,130583,NaN,NaN,NaN
2018-09-04 15:50:00,2896.00,2896.25,2886.25,2886.50,19416,144627,NaN,NaN,NaN
2018-09-04 16:18:00,2886.75,2898.25,2886.50,2896.50,17426,139397,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2018-09-28 17:59:00,2921.50,2925.50,2921.25,2921.75,16542,138491,-0.085558,4.647197,-0.018411
2018-09-28 19:42:00,2921.50,2923.75,2915.50,2922.75,16539,135231,1.971203,4.620693,0.426603
2018-09-28 21:11:00,2922.75,2924.00,2917.50,2918.25,15378,133601,2.318393,4.613825,0.502488


In [38]:
vols.head(50)

date
2018-09-04 02:31:00   NaN
2018-09-04 13:31:00   NaN
2018-09-04 15:34:00   NaN
2018-09-04 15:50:00   NaN
2018-09-04 16:18:00   NaN
2018-09-04 16:51:00   NaN
2018-09-04 17:41:00   NaN
2018-09-04 18:23:00   NaN
2018-09-04 19:40:00   NaN
2018-09-04 21:09:00   NaN
2018-09-04 21:58:00   NaN
2018-09-05 09:49:00   NaN
2018-09-05 15:16:00   NaN
2018-09-05 15:49:00   NaN
2018-09-05 16:12:00   NaN
2018-09-05 16:26:00   NaN
2018-09-05 16:50:00   NaN
2018-09-05 17:28:00   NaN
2018-09-05 18:21:00   NaN
2018-09-05 19:33:00   NaN
2018-09-05 21:02:00   NaN
2018-09-05 21:56:00   NaN
2018-09-06 03:50:00   NaN
2018-09-06 14:34:00   NaN
2018-09-06 15:46:00   NaN
2018-09-06 16:13:00   NaN
2018-09-06 16:34:00   NaN
2018-09-06 16:56:00   NaN
2018-09-06 17:20:00   NaN
2018-09-06 18:00:00   NaN
2018-09-06 19:11:00   NaN
2018-09-06 20:32:00   NaN
2018-09-06 21:30:00   NaN
2018-09-06 21:59:00   NaN
2018-09-07 09:27:00   NaN
2018-09-07 14:54:00   NaN
2018-09-07 15:39:00   NaN
2018-09-07 16:01:00   NaN
2018-09

In [39]:
x = pd.Series({'a': 1, 'b': -2, 'c': 4, 'd': -7})
x

a    1
b   -2
c    4
d   -7
dtype: int64

In [40]:
x.apply(lambda x: max(x,0))

a    1
b    0
c    4
d    0
dtype: int64